In [2]:
import requests
from IPython.display import Image, display, clear_output
import time
from urllib.parse import urlencode

jetbot_ip = '194.47.156.221'
jetbot_ip_2 = '194.47.156.140'

# Function to set motors
def set_motors(jetbot_ip, left_speed, right_speed):
    params = {'left': left_speed, 'right': right_speed}
    url = f'http://{jetbot_ip}:8080/set_motors?{urlencode(params)}'
    response = requests.get(url)
    if response.status_code == 200:
        print("Motors set successfully")
    else:
        print("Failed to set motors")

# Function to execute left command
def move_left(jetbot_ip, speed):
    params = {'speed': speed}
    url = f'http://{jetbot_ip}:8080/left?{urlencode(params)}'
    response = requests.get(url)
    """
    if response.status_code == 200:
        print("Left command executed successfully")
    else:
        print("Failed to execute left command")
    """


# Function to execute right command
def move_right(jetbot_ip, speed):
    params = {'speed': speed}
    url = f'http://{jetbot_ip}:8080/right?{urlencode(params)}'
    response = requests.get(url)
    """
    if response.status_code == 200:
        print("Right command executed successfully")
    else:
        print("Failed to execute right command")
    """


# Function to execute forward command
def move_forward(jetbot_ip, speed):
    params = {'speed': speed}
    url = f'http://{jetbot_ip}:8080/forward?{urlencode(params)}'
    response = requests.get(url)
    """
    if response.status_code == 200:
        print("Forward command executed successfully")
    else:
        print("Failed to execute forward command")
    """


# Function to execute stop command
def stop_robot(jetbot_ip):
    url = f'http://{jetbot_ip}:8080/stop'
    response = requests.get(url)
    if response.status_code == 200:
        print("Stop command executed successfully")
    else:
        print("Failed to execute stop command")


# Function to display continuous camera stream
def display_camera_stream(jetbot_ip):
    while True:
        response = requests.get(f'http://{jetbot_ip}:8080/camera')
        image = Image(response.content)
        clear_output(wait=True)
        display(image)
        time.sleep(0.1)  # Adjust the sleep time to control the frame rate

In [47]:
move_forward(jetbot_ip, 0.1)

In [97]:
stop_robot(jetbot_ip)

KeyboardInterrupt: 

In [25]:
display_camera_stream(jetbot_ip)

TypeError: 'module' object is not callable

In [10]:
import cv2
import numpy as np
import requests
from PIL import Image
import io

jetbot_ip = '194.47.156.142'

# Parametrar
KNOWN_WIDTH = 3.5  # iPhone bredd i cm
FOCAL_LENGTH = 500  # Byt till kalibrerat värde!

def get_jetbot_frame(jetbot_ip):
    response = requests.get(f'http://{jetbot_ip}:8080/camera')
    image = Image.open(io.BytesIO(response.content))
    frame = cv2.cvtColor(np.array(image), cv2.COLOR_RGB2BGR)
    return frame

robot_not_finished = True
gather_start_distance = True

while robot_not_finished:
    try:
        frame = get_jetbot_frame(jetbot_ip)

        # LAB konvertering
        lab = cv2.cvtColor(frame, cv2.COLOR_BGR2LAB)
        L, A, B = cv2.split(lab)
        _, mask = cv2.threshold(A, 150, 255, cv2.THRESH_BINARY)

        kernel = np.ones((5,5), np.uint8)
        mask = cv2.morphologyEx(mask, cv2.MORPH_OPEN, kernel)
        mask = cv2.morphologyEx(mask, cv2.MORPH_DILATE, kernel)

        contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

        for contour in contours:
            area = cv2.contourArea(contour)
            if area > 15:
                x, y, w, h = cv2.boundingRect(contour)
                cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 3)

                # Avståndsberäkning
                distance_cm = (KNOWN_WIDTH * FOCAL_LENGTH) / w
                cv2.putText(frame, f"Distance: {distance_cm:.2f} cm", (x, y - 10),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0,255,0), 2)



                if (distance_cm >= 70):
                    if gather_start_distance == True:
                        start_distance = distance_cm
                        gather_start_distance = False
                    move_forward(jetbot_ip, 0.1)
                else:
                    stop_robot(jetbot_ip)
                    robot_not_finished = False
                    stopping_distance = distance_cm
                    
                    print("Start distance: ", start_distance)
                    print("Stopping distance: ", stopping_distance)

        # Visa
        cv2.imshow("Jetbot Red Object + Distance", frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    except Exception as e:
        print(f"Error: {e}")

cv2.destroyAllWindows()


ModuleNotFoundError: No module named 'cv2'

In [18]:
import cv2
import numpy as np
import requests
from PIL import Image
import io

jetbot_ip = '194.47.156.221'

# Parametrar
KNOWN_WIDTH = 3.5  # iPhone bredd i cm
FOCAL_LENGTH = 500  # Byt till kalibrerat värde!

def get_jetbot_frame(jetbot_ip):
    response = requests.get(f'http://{jetbot_ip}:8080/camera')
    image = Image.open(io.BytesIO(response.content))
    frame = cv2.cvtColor(np.array(image), cv2.COLOR_RGB2BGR)
    return frame

robot_not_finished = True
gather_start_distance = True

while robot_not_finished:
    try:
        frame = get_jetbot_frame(jetbot_ip_2)

        # LAB konvertering
        lab = cv2.cvtColor(frame, cv2.COLOR_BGR2LAB)
        L, A, B = cv2.split(lab)
        _, mask = cv2.threshold(A, 150, 255, cv2.THRESH_BINARY)

        kernel = np.ones((5,5), np.uint8)
        mask = cv2.morphologyEx(mask, cv2.MORPH_OPEN, kernel)
        mask = cv2.morphologyEx(mask, cv2.MORPH_DILATE, kernel)

        contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

        for contour in contours:
            area = cv2.contourArea(contour)
            if area > 15:
                x, y, w, h = cv2.boundingRect(contour)
                cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 3)

                # Avståndsberäkning
                distance_cm = (KNOWN_WIDTH * FOCAL_LENGTH) / w
                cv2.putText(frame, f"Distance: {distance_cm:.2f} cm", (x, y - 10),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0,255,0), 2)


        # Visa
        cv2.imshow("Jetbot Red Object + Distance", frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    except Exception as e:
        print(f"Error: {e}")

cv2.destroyAllWindows()

KeyboardInterrupt: 

In [108]:
def get_jetbot_frame(jetbot_ip):
    response = requests.get(f'http://{jetbot_ip}:8080/camera')
    image = Image.open(io.BytesIO(response.content))
    frame = cv2.cvtColor(np.array(image), cv2.COLOR_RGB2BGR)
    return frame

def start_robot_one():
    robot_not_finished = True
    gather_start_distance = True

    tolerance = 20  # Pixels from center where no turning is needed
    turn_speed = 0.1
    forward_speed = 0.1

    no_target_counter = 0
    no_target_threshold = 5  # frames to wait before spinning

    while robot_not_finished:
        try:
            frame = get_jetbot_frame(jetbot_ip)
            image_width = frame.shape[1]
            image_center_x = image_width / 2

            # LAB konvertering
            lab = cv2.cvtColor(frame, cv2.COLOR_BGR2LAB)
            L, A, B = cv2.split(lab)
            _, mask = cv2.threshold(A, 150, 255, cv2.THRESH_BINARY)

            kernel = np.ones((5,5), np.uint8)
            mask = cv2.morphologyEx(mask, cv2.MORPH_OPEN, kernel)
            mask = cv2.morphologyEx(mask, cv2.MORPH_DILATE, kernel)

            contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
            found_target = False

            for contour in contours:
                area = cv2.contourArea(contour)
                if area > 15:
                    found_target = True

                    x, y, w, h = cv2.boundingRect(contour)
                    cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 3)

                    # Avståndsberäkning
                    distance_cm = (KNOWN_WIDTH * FOCAL_LENGTH) / w
                    cv2.putText(frame, f"Distance: {distance_cm:.2f} cm", (x, y - 10),
                                cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0,255,0), 2)

                    target_x = x + w / 2  # Center of detected red object

                    if distance_cm >= 30:
                        if gather_start_distance:
                            start_distance = distance_cm
                            gather_start_distance = False

                        # Alignment logic
                        if abs(target_x - image_center_x) <= tolerance:
                            move_forward(jetbot_ip, forward_speed)
                        elif target_x < image_center_x - tolerance:
                            move_left(jetbot_ip, turn_speed)
                        elif target_x > image_center_x + tolerance:
                            move_right(jetbot_ip, turn_speed)
                    else:
                        stop_robot(jetbot_ip)
                        robot_not_finished = False
                        stopping_distance = distance_cm

                        print("Start distance: ", start_distance)
                        print("Stopping distance: ", stopping_distance)

            if not found_target:
                no_target_counter += 1
                if no_target_counter >= no_target_threshold:
                    print("Target lost — spinning to search...")
                    move_right(jetbot_ip, 0.2)  # Spin slowly in place
            else:
                no_target_counter = 0  # Reset if target was seen


            # Visa
            cv2.imshow("Jetbot Red Object + Distance", frame)
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break

        except Exception as e:
            print(f"Error: {e}")

    cv2.destroyAllWindows()


In [109]:
jetbot_ip_2 = '194.47.156.140'

def get_jetbot_frame(jetbot_ip):
    response = requests.get(f'http://{jetbot_ip}:8080/camera')
    image = Image.open(io.BytesIO(response.content))
    frame = cv2.cvtColor(np.array(image), cv2.COLOR_RGB2BGR)
    return frame

def start_robot_two():
    robot_not_finished = True
    gather_start_distance = True

    tolerance = 20  # Pixels from center where no turning is needed
    turn_speed = 0.1
    forward_speed = 0.1

    no_target_counter = 0
    no_target_threshold = 5  # frames to wait before spinning

    while robot_not_finished:
        try:
            frame = get_jetbot_frame(jetbot_ip_2)
            image_width = frame.shape[1]
            image_center_x = image_width / 2

            # LAB konvertering
            lab = cv2.cvtColor(frame, cv2.COLOR_BGR2LAB)
            L, A, B = cv2.split(lab)
            _, mask = cv2.threshold(A, 150, 255, cv2.THRESH_BINARY)

            kernel = np.ones((5,5), np.uint8)
            mask = cv2.morphologyEx(mask, cv2.MORPH_OPEN, kernel)
            mask = cv2.morphologyEx(mask, cv2.MORPH_DILATE, kernel)

            contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
            found_target = False

            for contour in contours:
                area = cv2.contourArea(contour)
                if area > 15:
                    found_target = True

                    x, y, w, h = cv2.boundingRect(contour)
                    cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 3)

                    # Avståndsberäkning
                    distance_cm = (KNOWN_WIDTH * FOCAL_LENGTH) / w
                    cv2.putText(frame, f"Distance: {distance_cm:.2f} cm", (x, y - 10),
                                cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0,255,0), 2)

                    target_x = x + w / 2  # Center of detected red object

                    if distance_cm >= 30:
                        if gather_start_distance:
                            start_distance = distance_cm
                            gather_start_distance = False

                        # Alignment logic
                        if abs(target_x - image_center_x) <= tolerance:
                            move_forward(jetbot_ip_2, forward_speed)
                        elif target_x < image_center_x - tolerance:
                            move_left(jetbot_ip_2, turn_speed)
                        elif target_x > image_center_x + tolerance:
                            move_right(jetbot_ip_2, turn_speed)
                    else:
                        stop_robot(jetbot_ip)
                        robot_not_finished = False
                        stopping_distance = distance_cm

                        print("Start distance: ", start_distance)
                        print("Stopping distance: ", stopping_distance)

            if not found_target:
                no_target_counter += 1
                if no_target_counter >= no_target_threshold:
                    print("Target lost — spinning to search...")
                    move_right(jetbot_ip_2, 0.2)  # Spin slowly in place
            else:
                no_target_counter = 0  # Reset if target was seen


            # Visa
            cv2.imshow("Jetbot Red Object + Distance", frame)
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break

        except Exception as e:
            print(f"Error: {e}")

    cv2.destroyAllWindows()

In [93]:
cv2.destroyAllWindows()

In [100]:
jetbot_ip_2 = '194.47.156.140'

In [114]:
start_robot_one()
start_robot_two()

KeyboardInterrupt: 

In [ ]:
import cv2
import numpy as np
import requests
from PIL import Image
import io

jetbot_ips = [jetbot_ip, jetbot_ip_2]

# States per robot
states = {
    jetbot_ip: {
        "robot_not_finished": True,
        "gather_start_distance": True,
        "no_target_counter": 0
    },
    jetbot_ip_2: {
        "robot_not_finished": True,
        "gather_start_distance": True,
        "no_target_counter": 0
    }
}

# Parametrar
KNOWN_WIDTH = 3.5  # iPhone bredd i cm
FOCAL_LENGTH = 500  # Byt till kalibrerat värde!

no_target_threshold = 5
tolerance = 30
turn_speed = 0.1
forward_speed = 0.2

def get_jetbot_frame(jetbot_ip):
    response = requests.get(f'http://{jetbot_ip}:8080/camera')
    image = Image.open(io.BytesIO(response.content))
    frame = cv2.cvtColor(np.array(image), cv2.COLOR_RGB2BGR)
    return frame

while any(state["robot_not_finished"] for state in states.values()):
    for ip in jetbot_ips:
        try:
            if not states[ip]["robot_not_finished"]:
                continue  # Skip if this bot is done

            frame = get_jetbot_frame(ip)
            image_width = frame.shape[1]
            image_center_x = image_width / 2

            # Image preprocessing
            lab = cv2.cvtColor(frame, cv2.COLOR_BGR2LAB)
            _, A, _ = cv2.split(lab)
            _, mask = cv2.threshold(A, 150, 255, cv2.THRESH_BINARY)
            kernel = np.ones((5, 5), np.uint8)
            mask = cv2.morphologyEx(mask, cv2.MORPH_OPEN, kernel)
            mask = cv2.morphologyEx(mask, cv2.MORPH_DILATE, kernel)

            contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

            found_target = False

            for contour in contours:
                if cv2.contourArea(contour) > 15:
                    found_target = True
                    x, y, w, h = cv2.boundingRect(contour)
                    distance_cm = (KNOWN_WIDTH * FOCAL_LENGTH) / w
                    target_x = x + w / 2
                    alignment_error = target_x - image_center_x

                    if distance_cm >= 40:
                        if states[ip]["gather_start_distance"]:
                            states[ip]["start_distance"] = distance_cm
                            states[ip]["gather_start_distance"] = False

                        if abs(alignment_error) <= tolerance:
                            move_forward(ip, forward_speed)
                        else:
                            if alignment_error < 0:
                                move_left(ip, turn_speed)
                            else:
                                move_right(ip, turn_speed)
                    else:
                        stop_robot(ip)
                        states[ip]["robot_not_finished"] = False
                        states[ip]["stopping_distance"] = distance_cm
                        print(f"[{ip}] Start distance: {states[ip]['start_distance']}")
                        print(f"[{ip}] Stopping distance: {states[ip]['stopping_distance']}")
                    break  # Only process the largest/first valid contour

            if not found_target:
                states[ip]["no_target_counter"] += 1
                if states[ip]["no_target_counter"] >= no_target_threshold:
                    print(f"[{ip}] Target lost — spinning to search...")
                    move_right(ip, 0.15)
            else:
                states[ip]["no_target_counter"] = 0

            # Show debug window (optional)
            # cv2.imshow(f"Jetbot {ip}", frame)

        except Exception as e:
            print(f"[{ip}] Error: {e}")

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cv2.destroyAllWindows()

[194.47.156.221] Target lost — spinning to search...
[194.47.156.140] Target lost — spinning to search...
[194.47.156.221] Target lost — spinning to search...
[194.47.156.140] Target lost — spinning to search...
[194.47.156.221] Target lost — spinning to search...
[194.47.156.140] Target lost — spinning to search...
[194.47.156.221] Target lost — spinning to search...
[194.47.156.140] Target lost — spinning to search...
[194.47.156.221] Target lost — spinning to search...
[194.47.156.140] Target lost — spinning to search...
[194.47.156.221] Target lost — spinning to search...
[194.47.156.140] Target lost — spinning to search...
[194.47.156.221] Target lost — spinning to search...
[194.47.156.140] Target lost — spinning to search...
[194.47.156.221] Target lost — spinning to search...
[194.47.156.140] Target lost — spinning to search...
[194.47.156.221] Target lost — spinning to search...
[194.47.156.140] Target lost — spinning to search...
[194.47.156.221] Target lost — spinning to sea

KeyboardInterrupt: 

In [11]:
jetbot_ips = [jetbot_ip, jetbot_ip_2]

robot_not_finished = {ip: True for ip in jetbot_ips}
gather_start_distance = {ip: True for ip in jetbot_ips}
no_target_counter = {ip: 0 for ip in jetbot_ips}
start_distance = {}
stopping_distance = {}

tolerance = 20  # Pixels from center where no turning is needed
turn_speed = 0.1
forward_speed = 0.1
no_target_threshold = 5  # frames to wait before spinning

while any(robot_not_finished.values()):
    for ip in jetbot_ips:
        if not robot_not_finished[ip]:
            continue  # Skip if this bot is already done

        try:
            frame = get_jetbot_frame(ip)
            image_width = frame.shape[1]
            image_center_x = image_width / 2

            # LAB konvertering
            lab = cv2.cvtColor(frame, cv2.COLOR_BGR2LAB)
            L, A, B = cv2.split(lab)
            _, mask = cv2.threshold(A, 150, 255, cv2.THRESH_BINARY)

            kernel = np.ones((5,5), np.uint8)
            mask = cv2.morphologyEx(mask, cv2.MORPH_OPEN, kernel)
            mask = cv2.morphologyEx(mask, cv2.MORPH_DILATE, kernel)

            contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
            found_target = False

            for contour in contours:
                area = cv2.contourArea(contour)
                if area > 15:
                    found_target = True

                    x, y, w, h = cv2.boundingRect(contour)
                    cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 3)

                    # Avståndsberäkning
                    distance_cm = (KNOWN_WIDTH * FOCAL_LENGTH) / w
                    cv2.putText(frame, f"Distance: {distance_cm:.2f} cm", (x, y - 10),
                                cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0,255,0), 2)

                    target_x = x + w / 2  # Center of detected red object

                    if distance_cm >= 30:
                        if gather_start_distance[ip]:
                            start_distance[ip] = distance_cm
                            gather_start_distance[ip] = False

                        # Alignment logic
                        if abs(target_x - image_center_x) <= tolerance:
                            move_forward(ip, forward_speed)
                        elif target_x < image_center_x - tolerance:
                            move_left(ip, turn_speed)
                        elif target_x > image_center_x + tolerance:
                            move_right(ip, turn_speed)
                    else:
                        stop_robot(ip)
                        robot_not_finished[ip] = False
                        stopping_distance[ip] = distance_cm

                        print(f"[{ip}] Start distance: ", start_distance[ip])
                        print(f"[{ip}] Stopping distance: ", stopping_distance[ip])
                    break  # Only process the first valid contour

            if not found_target:
                no_target_counter[ip] += 1
                if no_target_counter[ip] >= no_target_threshold:
                    print(f"[{ip}] Target lost — spinning to search...")
                    move_right(ip, 0.2)
            else:
                no_target_counter[ip] = 0

            # Optional: show camera feed from each robot
            # cv2.imshow(f"Jetbot {ip}", frame)

        except Exception as e:
            print(f"[{ip}] Error: {e}")

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cv2.destroyAllWindows()


[194.47.156.221] Target lost — spinning to search...
[194.47.156.140] Target lost — spinning to search...
[194.47.156.221] Target lost — spinning to search...
[194.47.156.140] Target lost — spinning to search...
[194.47.156.221] Target lost — spinning to search...
[194.47.156.140] Target lost — spinning to search...
[194.47.156.221] Target lost — spinning to search...
[194.47.156.140] Target lost — spinning to search...
[194.47.156.221] Target lost — spinning to search...
[194.47.156.140] Target lost — spinning to search...
[194.47.156.221] Target lost — spinning to search...
[194.47.156.140] Target lost — spinning to search...
[194.47.156.221] Target lost — spinning to search...
[194.47.156.140] Target lost — spinning to search...
[194.47.156.221] Target lost — spinning to search...
[194.47.156.140] Target lost — spinning to search...
[194.47.156.221] Target lost — spinning to search...
[194.47.156.140] Target lost — spinning to search...
[194.47.156.221] Target lost — spinning to sea

KeyboardInterrupt: 

In [103]:
move_forward(jetbot_ip, 0.1)
move_forward(jetbot_ip_2, 0.1)

In [21]:
stop_robot(jetbot_ip)


Stop command executed successfully
